# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [1]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [2]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [6]:
show(first(boston,1),allcols=true)

1×14 DataFrame
│ Row │ Crim    │ Zn      │ Indus   │ Chas  │ NOx     │ Rm      │ Age     │
│     │ Float64 │ Float64 │ Float64 │ Int64 │ Float64 │ Float64 │ Float64 │
├─────┼─────────┼─────────┼─────────┼───────┼─────────┼─────────┼─────────┤
│ 1   │ 0.00632 │ 18.0    │ 2.31    │ 0     │ 0.538   │ 6.575   │ 65.2    │

│ Row │ Dis     │ Rad   │ Tax   │ PTRatio │ Black   │ LStat   │ MedV    │
│     │ Float64 │ Int64 │ Int64 │ Float64 │ Float64 │ Float64 │ Float64 │
├─────┼─────────┼───────┼───────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ 4.09    │ 1     │ 296   │ 15.3    │ 396.9   │ 4.98    │ 24.0    │

In [7]:
boston[!, :LogMedV] = log.(boston[!, :MedV])

506-element Array{Float64,1}:
 3.1780538303479458
 3.0726933146901194
 3.5467396869528134
 3.5085558999826545
 3.5890591188317256
 3.3568971227655755
 3.131136910560194 
 3.299533727885655 
 2.803360380906535 
 2.9391619220655967
 2.70805020110221  
 2.9391619220655967
 3.077312260546414 
 ⋮                 
 3.1986731175506815
 3.139832617527748 
 2.9806186357439426
 2.9069010598473755
 3.054001181677967 
 2.8622008809294686
 2.8213788864092133
 3.109060958860994 
 3.0252910757955354
 3.173878458937465 
 3.091042453358316 
 2.4765384001174837

In [8]:
indecies=MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind,test_ind = MLDataUtils.splitobs(indecies, at=0.8);

In [9]:
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.1712,0.0,8.56,0,0.52,5.836,91.9,2.211,5,384
2,7.83932,0.0,18.1,0,0.655,6.209,65.4,2.9634,24,666
3,5.20177,0.0,18.1,1,0.77,6.127,83.4,2.7227,24,666
4,4.42228,0.0,18.1,0,0.584,6.003,94.5,2.5403,24,666
5,17.8667,0.0,18.1,0,0.671,6.223,100.0,1.3861,24,666
6,0.11329,30.0,4.93,0,0.428,6.897,54.3,6.3361,6,300
7,0.01311,90.0,1.22,0,0.403,7.249,21.9,8.6966,5,226
8,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
9,0.03041,0.0,5.19,0,0.515,5.895,59.6,5.615,5,224


In [10]:
model = lm(@formula(LogMedV ~ Crim + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

LogMedV ~ 1 + Crim + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
──────────────────────────────────────────────────────────────────────────────────────
                Estimate   Std. Error    t value  Pr(>|t|)     Lower 95%     Upper 95%
──────────────────────────────────────────────────────────────────────────────────────
(Intercept)   4.0505      0.228341      17.7388     <1e-51   3.60158       4.49941    
Crim         -0.00829392  0.00184194    -4.50282    <1e-5   -0.0119152    -0.00467266 
Chas          0.122803    0.0414186      2.96492    0.0032   0.0413737     0.204232   
NOx          -0.75765     0.160862      -4.70993    <1e-5   -1.07391      -0.441394   
Rm            0.0946156   0.0183685      5.15097    <1e-6    0.0585031     0.130728   
Dis          -0.0463178   0.00737157  

In [11]:
exp.(predict(model, test))

101-element Array{Float64,1}:
 16.49899022624287 
 18.96583036131898 
 21.28340587402965 
 15.49401027539447 
 14.15191416110422 
 25.00726678454423 
 28.333073110179768
 28.39543678299243 
 20.220337499914237
 20.77683041989934 
 23.23364545203103 
 19.894994654000826
 39.95917298937685 
  ⋮                
 17.20405032167582 
 25.47292342432617 
 22.65791179412269 
 33.42025566451017 
 16.33642168493491 
 31.282694278930823
 44.29403257346235 
 35.675511466781344
 18.28665277126353 
 35.817802320168   
 37.72007982713689 
 22.35297869498835 